# base_rbt -- Basic functions/classes needed for RBT

> Basic stuff we need

In [ ]:
#| default_exp base_rbt

In [ ]:
#| export
import torch
if torch.cuda.is_available():device='cuda'
else:device='cpu'
import random
import os
import numpy as np
import math

In [ ]:
#| export
def seed_everything(seed=42):
    """"
    Seed everything.
    """   
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

In [ ]:
#| export

def random_sinusoid(x:torch.Tensor,std=0.1,seed=0):
    """Input a tensor x of shape: batch_size * dim. Output random sinusoid applied pointwise
    to x. i.e. if x has shape bs*d, implicitly sample d random sinusoids and apply along batch
    dimension
    """

    dim=x.shape[1]
    seed_everything(seed=seed)    
    t=(std) * torch.randn(1,dim).to(device)
    s=(std) * torch.randn(1,dim).to(device)
    
    u=torch.randn(1,dim).to(device)
    v=torch.randn(1,dim).to(device)

    a=(0.2) * torch.randn(1,dim).to(device)
    b=(0.2) * torch.randn(1,dim).to(device)

    return a*torch.sin(t*x[:,]*math.pi+u) + b*torch.cos(s*x[:,]*math.pi+v)

#Uniform random number between a and b
def Unif(a,b):
    return (b-a)*torch.rand(1).item()+a

def C_z1z2(z1norm,z1norm_2,z2norm,z2norm_2,bs):
    "Compute 0.5*corr(z1norm,z2norm_2) + 0.5*corr(z2norm_2,z2norm)"
    
    Ctem1 =  (z1norm.T @ z2norm_2) / bs
    Ctem2 = (z1norm_2.T @ z2norm) / bs
    cdiff_2 = (0.5*Ctem1.pow(2) + 0.5*Ctem2.pow(2))

    return cdiff_2

In [ ]:
#| export

class Cdiff_Rand:
    
    def __init__(self,seed,bs,std=0.1,K=2):
        self.seed=seed
        self.std=std
        self.K=2
        self.bs=bs

    def __call__(self,z1norm,z2norm):
        
        cdiff_rand=0
        for i in range(self.K):

            z1norm_2,z2norm_2 = random_sinusoid(z1norm,std=self.std,seed=self.seed+i), random_sinusoid(z2norm,std=self.std,seed=2*self.seed+i)
            cdiff_rand = C_z1z2(z1norm=z1norm,z1norm_2=z1norm_2,z2norm=z2norm,z2norm_2=z2norm_2,bs=bs)

        cdiff_rand=(1/self.K)*cdiff_rand
    
        return cdiff_rand

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()